# Setup SSH keys for GitHub

This notebook helps you setup a unique SSH keypair to allow you to check out private repos both into the Jupyter container and into your VMs. It is intended to help get the code out of *private* GitHub repos. For public repos `https://` access schema is used wherever possible. 

## Overview

- Generate a new SSH keypair and place it under ~/work/fabric_config/github_ecdsa[.pub]
- Provide instructions on using the keypair for checking out the code locally into the notebook and in the VMs

## Generate SSH keypair

In [ ]:
# replace with the email associated with your GitHub account
export MY_GITHUB_EMAIL='someone@email.com'

# name of the keyfile (public key will have .pub extension)
GITHUB_KEY=~/work/fabric_config/github_ecdsa

# check if keys exist already
if [ -e ${GITHUB_KEY} ]; then
    echo "Key already exists don't proceed unless you are sure, old key will be saved with .bak extension."
    echo "Existing key signatures:"
    # MD5 signature of the key
    ssh-keygen -E md5 -lf ${GITHUB_KEY}

    # SHA256 signature of the key
    ssh-keygen -E sha256 -lf ${GITHUB_KEY}
else
    echo "No key found, good to proceed"
fi

In [ ]:
# remove old key files
if [ -e ${GITHUB_KEY} ]; then
    echo "Moving the old keys"
    mv ${GITHUB_KEY} ${GITHUB_KEY}.bak
    mv ${GITHUB_KEY}.pub ${GITHUB_KEY}.bak.pub
fi

# generate a new ECDSA key with no passprase
ssh-keygen -t ecdsa -C ${MY_GITHUB_EMAIL} -f ~/work/fabric_config/github_ecdsa -N ''

In [ ]:
# show the generated keys and their signatures
ls -l ${GITHUB_KEY} ${GITHUB_KEY}.pub

# MD5 signature of the key
ssh-keygen -E md5 -lf ${GITHUB_KEY}

# SHA256 signature of the key
ssh-keygen -E sha256 -lf ${GITHUB_KEY}

## Register the public key with GitHub

Visit [GitHub](https://github.com/settings/keys) key settings and add the following public key as a new Authentication Key with a title 'FABRIC SSH Key' or another appropriate title:

<div>
    <img src="figs/add-key.png" width=500>
</div>


In [ ]:
# display the public key so you can cut and paste it into the browser

cat ${GITHUB_KEY}.pub

## Using the key to checkout the code

Normally you would add this key to your SSH configuration and use it with ssh-agent, however since this key has no passphrase and is meant to be used in multiple hosts, the easiest way is to use it like the following command shows (it can be used for cloning or commits [although if code signing is enabled you would also need the signing key]):

```
$ GIT_SSH_COMMAND='ssh -i ~/work/fabric_config/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone git@github.com:JeffersonLab/E2SAR.git
```

when updating existing code you can use
```
$ GIT_SSH_COMMAND='ssh -i ~/work/fabric_config/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git pull origin
```

## Fix permissions of the keys

When you start your container again, your private SSH key created above may have the wrong permissions (this is a property of GitHub workspace disk), the following command will fix them:

In [ ]:
# fix key permissions

chmod go-rwx ~/work/fabric_config/github_ecdsa
